# 5. cvičení - High Dynamic Range (HDR)
[High Dynamic Range Imaging](https://en.wikipedia.org/wiki/High-dynamic-range_imaging) je technika umožňující rozšířenou reprodukci rozsahu osvětlení a zastínění částí scény snímku mezi nejsvětlejším a nejtmavším bodem snímku, než je možné u normální snímací techniky bez tohoto postupu.

In [ ]:
import exif
import cv2
import matplotlib.pyplot as plt
import glob
import numpy as np
from copy import copy

In [ ]:
def read_images(file_pattern, scale_percent = 5):
    files = glob.glob(file_pattern) # Načtení cest ke všem souborům dle daného vzoru

    # příprava listů
    imgs = []
    t = []

    # cyklus přes všechny soubory (obrázky v dané cestě)
    for file in files:
        tmp_img = cv2.imread(file) # Načtení obrázku
        width = int(tmp_img.shape[1] * scale_percent / 100) #nový rozměr po zmenšení dle dané hodnoty v %
        height = int(tmp_img.shape[0] * scale_percent / 100) #nový rozměr po zmenšení dle dané hodnoty v %
        dim = (width, height) #nový rozměr po zmenšení dle dané hodnoty v %
        imgs.append(cv2.resize(tmp_img, dim, interpolation = cv2.INTER_AREA)) # zmenšení obrázku a uložení do listu
        info = exif.Image(file) # Načtení EXIF z obrázku
        t.append(info.exposure_time) # Uložení času expozice do listu
    t = np.array(t)
    return imgs, t # Návrat velikosti obrázků, pole s obrázky a vektoru časů expozic


def bgr2rgb(bgr_image):
    b,g,r = cv2.split(bgr_image) # Rozdělení barevných kanálů
    return cv2.merge([r,g,b]) # Spojení v jiném pořadí a použití jako návratovou hodnotu funkce

In [ ]:
images, t = read_images("./images/*.jpg", 20) #Načtení cest obrázků a časů expozic

In [ ]:
for idx in range(len(t)):
    plt.figure()
    plt.imshow(bgr2rgb(images[idx]))
    plt.title('{} s'.format(t[idx]))

### Spojení expozci do HDR obrázku pomocí OpenCV

In [ ]:
# metoda Martens
merge_martens = cv2.createMergeMertens()
hdr_martens = merge_martens.process(images)

In [ ]:
hdr = hdr_martens - np.min(hdr_martens)
hdr = hdr/np.max(hdr)
plt.imshow(bgr2rgb(hdr))


In [ ]:
tonemap = cv2.createTonemap(gamma=1)
res_martens = tonemap.process(hdr_martens.copy())
plt.imshow(bgr2rgb(res_martens))

### Vyrovnání histogramu

In [ ]:
res_martens_uint8 = cv2.normalize(res_martens, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
res_martens_ycrcb = cv2.cvtColor(res_martens_uint8, cv2.COLOR_BGR2YCrCb)
y,cr,cb = cv2.split(res_martens_ycrcb)
equ_y = cv2.equalizeHist(y)
equ_ycrcb = cv2.merge([equ_y, cr, cb])
equ = cv2.cvtColor(equ_ycrcb, cv2.COLOR_YCrCb2RGB)
plt.imshow(equ)

## Řídké matice

Práci s řídkými maticemi umožňuje například knihvna scipy, konkrétně [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html). K řešení je vhodný objekt [Compressed Sparse Row matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix)

In [ ]:
from scipy.sparse import csr_matrix, hstack, vstack
row = np.array([0, 1, 2, 0])
col = np.array([0, 1, 1, 0])
data = np.array([1, 2, 4, 8])

csr_matrix((data, (row, col)), shape=(3, 3)).toarray()

Pokud potřebujeme například složit dvě matice pak k tomu složí metody hstack (horizontální) a vstack (vertikální)

In [ ]:
spr = csr_matrix((data, (row, col)), shape=(3, 3))

In [ ]:
hstack([spr, spr]).toarray()

In [ ]:
vstack([spr, spr]).toarray()

Pro řešení předefinovaných soustav rovnic definovaných řídkými maticemi pomocí metody nejmenších čtverců odchylek souží funkce [scipy.sparse.linalg.lsqr](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lsqr.html#scipy.sparse.linalg.lsqr)

In [ ]:
from scipy.sparse.linalg import lsqr
A = csr_matrix([[1., 0.], [1., 1.], [0., 1.]], dtype=float)
b = np.array([1., 0.01, -1.], dtype=float)
sol = lsqr(A, b)

In [ ]:
sol